In [810]:
## Import lib for extracting training set from pdfs
import extract
import importlib
importlib.reload(extract)
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [842]:
#  Extract training set
train_indicies = extract.extract_words()

In [843]:
# Read training files as one string
smr_content = open('sum_train.txt', 'r').read()
xprnc_content = open('exp_train.txt', 'r').read()
edct_content = open('edu_train.txt', 'r').read()

In [813]:
def string_list(s_term, s_str):
    result = []
    crnt_ptr = s_str.find(s_term) + len(s_term)
    scd_term_index=s_str.find(s_term, crnt_ptr)
    while (scd_term_index>=0):
        # count only significantly long segments (>9 characters)
        if (scd_term_index-crnt_ptr>9):
            result.append(s_term + ' ' + s_str[crnt_ptr:scd_term_index])
        crnt_ptr = scd_term_index + len(s_term)
        scd_term_index=s_str.find(s_term, crnt_ptr)
    return result

In [844]:
# Put segment from each CV into separate string in an array
sum_list = string_list("Summary", smr_content)
xprnc_list = string_list("Experience", xprnc_content)
edctn_list = string_list("Education", edct_content)

##
(jan(uary|\.|)|feb(ruary|\.|)|mar(ch|\.|)|apr(il|\.|)|may(\.|)|jun(e|\.|)|jul(y|\.|)|aug(ust|\.|)|sep(tember|\.|)|oct(ober|\.|)|nov(ember|\.|)|dec(ember|\.|))\s+\d\d*

In [815]:
# Function for extracting vocabulary specific for a segment
import re
def create_dictionary(topic_list):
    month_expr = re.compile(r"(jan(uary|\.|)|feb(ruary|\.|)|mar(ch|\.|)|apr(il|\.|)|may(\.|)|jun(e|\.|)|jul(y|\.|)|aug(ust|\.|)|sep(tember|\.|)|oct(ober|\.|)|nov(ember|\.|)|dec(ember|\.|))")
    year_expr = re.compile(r"((19|20)\d\d|'?\d\d)")
    result = dict()
    for string in topic_list:
        a = re.split('[^0-9A-Za-z]', string)
        a=list(filter(lambda x: x!='', a))
        for i in range(len(a)):
            a[i] = a[i].casefold()
            if re.fullmatch(year_expr, a[i]):
                a[i] = '!year'
            if re.fullmatch(month_expr, a[i]):
                a[i] = '!month'
            if (a[i] in result):
                result[a[i]] += 1
            else:
                result[a[i]] = 1
                
    return result


In [845]:
# Extract vocabulary specific for a segment
sum_dct = create_dictionary(sum_list)
xprnc_dct = create_dictionary(xprnc_list)
edctn_dct = create_dictionary(edctn_list)

In [817]:
# Functions for calculating term frequencies and corpus dictionary creation
def concat_dct(d1, d2):
    d3 = dict(d1)
    for key in d2.keys():
        if (key in d3):
            d3[key] += d2[key]
        else:
            d3[key] = d2[key]
    return d3

def sumDictValues(dctnr):
    N = 0
    for val in dctnr.values():
        N = + N + val
    return N

def calc_probs(dctnr):
    prob_dctnr = dict()
    tot_size = sumDictValues(dctnr)
    for key in dctnr.keys():
        prob_dctnr[key] = dctnr[key]/tot_size
    return prob_dctnr

In [846]:
# Uniting into corpus dictionary
new_dict = concat_dct(xprnc_dct, edctn_dct)
whole_dict = concat_dct(new_dict, sum_dct)

In [847]:
# Calculating probabilities
sum_prb = calc_probs(sum_dct)
xprnc_prb = calc_probs(xprnc_dct)
edctn_prb = calc_probs(edctn_dct)
whole_dict_prb = calc_probs(whole_dict)

months = ['']
def replace_dates(a):
    for i 

In [848]:
# Test data generation
smr_test_content = open('sum_test.txt', 'r').read()
exp_test_content = open('exp_test.txt', 'r').read()
edu_test_content = open('edu_test.txt', 'r').read()

In [821]:
def replace_with_space(matchobj):
    return " "

def construct_marked_set(new_content):
    CRIT_LENGTH_LIMIT = 100
    result = np.zeros(4).reshape(1,4)
#    rep_pattern = re.compile(r"((Summary|Experience|Education)|[^A-Za-z0-9\s])")
    rep_pattern = re.compile(r"([^A-Za-z0-9\s])")
    category = 1
    for string in new_content:
        a = re.sub(rep_pattern, replace_with_space, string)
        cumulative_str = ''
        for substr in a.split('\n'):
            cumulative_str += substr + ' '
            if(len(cumulative_str) > CRIT_LENGTH_LIMIT):
                result = np.append(result, np.asarray([cumulative_str, category, 0, 0]).reshape(1,4), axis=0)
                cumulative_str = ''
        if(len(cumulative_str) > 2):
                result = np.append(result, np.asarray([cumulative_str, category, 0, 0]).reshape(1,4), axis=0)
        category += 1
    return result[1:]

In [862]:
test_set = construct_marked_set([smr_test_content, edu_test_content, exp_test_content])

In [861]:
def calculate_likelihood(string, dct, corp_dct, prb_segment):
    month_expr = re.compile(r"(jan(uary|\.|)|feb(ruary|\.|)|mar(ch|\.|)|apr(il|\.|)|may(\.|)|jun(e|\.|)|jul(y|\.|)|aug(ust|\.|)|sep(tember|\.|)|oct(ober|\.|)|nov(ember|\.|)|dec(ember|\.|))")
    year_expr = re.compile(r"((19|20)\d\d|'?\d\d)")
    num = 1
    den = 1
    for word in string.split():
        wrd = word.casefold()
        if re.fullmatch(year_expr, wrd):
                wrd = '!year'
        if re.fullmatch(month_expr, wrd):
                wrd = '!month'
        #print(wrd)
        if wrd in dct:
            num *= dct[wrd]
            den *= corp_dct[wrd]
    #print(num)
    #print(den)
    return prb_segment*num/den
    
def test_algorithm(test_set, dictionaries, corpus_dict):
    #likelihoods = np.zeros(len(dictionaries))
    prb_corpus = calc_probs(corpus_dict)
    i=0
    for a_dict in dictionaries:
            prb_segment = sumDictValues(a_dict) / sumDictValues(corpus_dict)
            print(prb_segment)
            prb_dict = calc_probs(a_dict)
            for line in test_set:
#                print('new_like')
#                print(line[0] + "ttteeest")
                likelihood = calculate_likelihood(line[0], prb_dict, prb_corpus, prb_segment)
                if likelihood > np.float(line[3]):
                    line[3] = likelihood
                    line[2] = i+1
            i+=1

In [708]:
calculate_likelihood(test_set[78][0], xprnc_prb, whole_dict_prb)

TypeError: calculate_likelihood() missing 1 required positional argument: 'prb_segment'

In [824]:
sumDictValues(sum_dct) + sumDictValues(edctn_dct) + sumDictValues(xprnc_dct)

17970

In [825]:
sumDictValues(whole_dict)

17970

In [863]:
test_algorithm(test_set, [sum_dct, edctn_dct, xprnc_dct], whole_dict)

0.1635442172996982
0.14458174363646717
0.6918740390638346


In [864]:
print(classification_report(test_set[:,1], test_set[:,2], target_names=['smr', 'edu', 'xp']))

             precision    recall  f1-score   support

        smr       0.40      0.81      0.53        42
        edu       0.76      1.00      0.86        41
         xp       0.97      0.65      0.78       171

avg / total       0.84      0.73      0.75       254



In [852]:
print(confusion_matrix(test_set[:,1], test_set[:,2]))

[[ 34   5   3]
 [  0  41   0]
 [ 52   8 111]]


In [828]:
(test_set[:,1] != test_set[:,2]).sum()

89

In [856]:
test_set[:,2]=='3'

array([False,  True, False, False, False, False, False,  True, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True, False, False, False,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,

In [834]:
test_set[:,2]

array(['3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3

In [809]:
term = 'experience'
print(sum_prb[term])
print(xprnc_prb[term])
print(edctn_prb[term])


0.005303030303030303
0.005918660691637778


KeyError: 'experience'

In [682]:
print(sumDictValues(whole_dict))

18352


In [690]:
np.sort(list(whole_dict.values()))

array([  1,   1,   1, ..., 481, 752, 954])

In [689]:
list(whole_dict.values())

[204,
 175,
 312,
 16,
 481,
 954,
 81,
 27,
 192,
 1,
 24,
 1,
 6,
 26,
 19,
 1,
 1,
 230,
 1,
 16,
 6,
 3,
 11,
 752,
 5,
 8,
 21,
 9,
 48,
 56,
 97,
 37,
 76,
 48,
 149,
 15,
 144,
 1,
 1,
 361,
 10,
 36,
 298,
 11,
 15,
 20,
 2,
 1,
 26,
 1,
 1,
 110,
 33,
 170,
 54,
 410,
 4,
 7,
 2,
 5,
 43,
 6,
 43,
 1,
 14,
 18,
 20,
 14,
 30,
 8,
 1,
 1,
 1,
 5,
 8,
 1,
 50,
 29,
 30,
 22,
 10,
 22,
 47,
 3,
 14,
 35,
 2,
 32,
 1,
 1,
 36,
 3,
 4,
 3,
 1,
 2,
 1,
 3,
 7,
 8,
 27,
 19,
 7,
 16,
 2,
 8,
 9,
 1,
 4,
 14,
 14,
 12,
 22,
 151,
 3,
 23,
 8,
 4,
 32,
 2,
 31,
 1,
 39,
 1,
 7,
 7,
 3,
 7,
 1,
 1,
 263,
 13,
 14,
 10,
 25,
 1,
 10,
 3,
 55,
 109,
 3,
 1,
 12,
 4,
 8,
 1,
 114,
 121,
 5,
 1,
 1,
 1,
 1,
 1,
 7,
 8,
 2,
 1,
 22,
 1,
 1,
 12,
 2,
 62,
 6,
 7,
 30,
 2,
 5,
 17,
 11,
 80,
 14,
 8,
 45,
 52,
 94,
 10,
 4,
 33,
 2,
 1,
 4,
 1,
 5,
 75,
 53,
 13,
 62,
 1,
 9,
 10,
 1,
 1,
 4,
 1,
 1,
 1,
 7,
 1,
 1,
 9,
 12,
 11,
 3,
 8,
 81,
 4,
 17,
 16,
 3,
 1,
 3,
 1,
 3,
 37,
 1,
 9,
 4,


In [635]:
len(edctn_prb.keys())

653

In [633]:
test_set[:,1:]

array([['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '3'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '2'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '1'],
       ['1', '2'],
       ['1', '2'],
       ['1', '1'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '1'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2', '2'],
       ['2',